<a href="https://colab.research.google.com/github/Snootic/dio.etl-colab/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [7]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[5175, 5176, 5177, 5178, 5179]


In [14]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 5175,
    "name": "Joao Gustavo",
    "account": {
      "id": 5507,
      "number": "785872-2",
      "agency": "919231",
      "balance": 5323.23,
      "limit": 10000.0
    },
    "card": {
      "id": 5034,
      "number": "7848 6278 7666 3692",
      "limit": 10000.0
    },
    "features": [],
    "news": [
      {
        "id": 9552,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "\"Jo\u00e3o Gustavo, n\u00e3o seja um gastador compulsivo! Aprenda a economizar e investir o seu dinheiro como eu fa\u00e7o!\" - Tio Patinhas"
      },
      {
        "id": 9555,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "\"Joao Gustavo, nao toque na minha caixa-forte, ou eu te mando para a Patagonia!\" - Tio Patinhas"
      }
    ]
  },
  {
    "id": 5176,
    "name": "Ostrudo Ostrado",
    "account": {
      "id": 5508,
    

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [4]:
!pip install g4f --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
# MINHA API DO GPT ACABOU ENTAO VOU USAR A API DO BING, QUE TAMBÉM USA O GPT-4 SÓ QUE DE GRAÇA
# TALVEZ ELA TENHA BIRRA COM O GOOGLE COLAB, POR CAUSA DO IP DOS SERVIDORES DA GOOGLE, PORTANTO PODE SER QUE A PRÓXIMA ETAPA DE ERRO AQUI
# PORÉM FUNCIONA EM MÁQUINA LOCAL, POIS O IP NÃO TA NA BLACKLIST DA MICROSOFT (VPN, SERVIDORES, SERVIÇOS NO GERAL)
# QUALQUER IP NUMA BLACKLIST PASSA PELO GEETEST, DAÍ TEM CAPTCHA, INFELIZMENTE NÃO TEM O QUE FAZER
# esqueci do caps lock

# bing é meio burro também, dependendo da resposta ele usa mais de 2 linhas, daí tem que rodar de novo para ficar tudo ok


In [9]:
import g4f
import re
from g4f.Provider import (
    Bing,
)

def generate_ai_news(user):
  response = g4f.ChatCompletion.create(
      model=g4f.models.default,
      messages=[{"role": "Você é o tio patinhas",
                "content": f"Você agora é o tio patinhas, crie uma frase que apenas ele diria para {user['name']} e termine com - Tio Patinhas (max de 150 caracteres e coloque entre aspas, não use nenhum acento)"}],
      provider=Bing,
      auth=True
  )
  return response


Exception: ignored

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"Usuário {user['name']} atualizado??? {success}!")

User Pyterson updated? True!
User Pip updated? True!
User Pep updated? True!
